In [2]:
import joblib
import pandas as pd
import numpy as np

In [4]:
lgbm_model = joblib.load('../output/lgbm_model.pkl')
data = pd.read_parquet('../data/ML_data/SA2_ML_data.parquet')

In [5]:
n = int(len(data) / 2)
pred = lgbm_model.predict(data.drop(['Station Name'], axis=1))
pred_weekday = pred[:n]
pred_weekend = pred[n:]

In [6]:
SA2 = list(data['Station Name'])[:n]
SA2_demand = pd.DataFrame([SA2, pred_weekday, pred_weekend], index=['SA2', 'log_demand_weekday', 'log_demand_weekend']).T

In [7]:
scaler = joblib.load('../data/ML_data/y_scaler.pickle')


In [8]:
SA2_demand['demand_weekday'] = scaler.inverse_transform(SA2_demand[['log_demand_weekday']])
SA2_demand['demand_weekday'] = SA2_demand['demand_weekday'].apply(lambda x: np.exp(x))

SA2_demand['demand_weekend'] = scaler.inverse_transform(SA2_demand[['log_demand_weekend']])
SA2_demand['demand_weekend'] = SA2_demand['demand_weekend'].apply(lambda x: np.exp(x))

In [9]:
SA2_demand['total_demand'] = (SA2_demand['demand_weekday'] * 5 + SA2_demand['demand_weekend'] * 2) / 7

In [11]:
SA2_demand.to_csv('../output/demand_predictions.csv', index=False)